# Neuroimaging 2025/2026
## Lab Introduction

**Tutors:** Beatriz Vale & Vânia Miguel

**Created by:** Marta Xavier & Neil Mehta

## Objectives

Introduce a suite of tools for Neuroimaging data including visualization options:
- Neurodesk
- FSL
- FSLeyes
- MRView
- Nipy

## I. Neurodesk

Neurodesk is an environment which comes with many classic Neuroimaging tools preinstalled.

By running the cell below (Shift+ENTER) in a Google Colab notebook, you install Neurodesk into that Colab instance.

**Note:** You are not required to run Neurodesk in a Colab environment.

In [ ]:
# Set up Neurodesk
%%capture
import os
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    os.environ["LD_PRELOAD"] = "";
    os.environ["APPTAINER_BINDPATH"] = "/content"
    os.environ["MPLCONFIGDIR"] = "/content/matplotlib-mpldir"
    os.environ["LMOD_CMD"] = "/usr/share/lmod/lmod/libexec/lmod"

    !curl -J -O https://raw.githubusercontent.com/neurodesk/neurocommand/main/googlecolab_setup.sh
    !chmod +x googlecolab_setup.sh
    !./googlecolab_setup.sh

    os.environ["MODULEPATH"] = ':'.join(map(str, list(map(lambda x: os.path.join(os.path.abspath('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/'), x), os.listdir('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/')))))

Neurodesk relies on the environment module manager lmod to load and unload different Neuroimaging tools. To see what tools are available, we can run the `lmod.avail()` command.

In [ ]:
# Download analysis packages
import lmod
await lmod.avail()

Since most of our labs will involve MRI data, we can try loading the FMRIB Software Library (FSL) and then try out a few commands.

In [ ]:
await lmod.load('fsl/6.0.7.16')

In [ ]:
# pwd shows us our present working directory.
# Note: we need to put '!' before any SHELL command for the interpreter to properly recognize it.
# If we need the effect of a command to persist after execution of the cell, we may replace '!' with '%'.
!pwd

In [ ]:
# Make your own directory
!mkdir test_dir

## II. FSL

In [ ]:
# After loading FSL using lmod, we can check what the environment variable, $FSLDIR, returns.
# This is the location of your FSL installation and contains some important data (such as brain atlases),
# we may use in later labs.
!echo $FSLDIR

In [ ]:
# Display the contents of the FSL data folder, containing different standard / atlas images
!ls $FSLDIR/data

In [ ]:
# Download some example datasets (from the FSL course) using the wget command.
!wget -c https://fsl.fmrib.ox.ac.uk/fslcourse/downloads/preCourse.tar.gz

In [ ]:
# Unpack the tar archive.
!tar xvf preCourse.tar.gz

In [ ]:
# Just to make sure, we can change all of the file
# permissions in the intro directory to read, write, execute
!chmod 777 -R fsl_course_data/intro

In [ ]:
# Change directory (there are 2 ways)
# %cd fsl_course_data/intro/
os.chdir('fsl_course_data/intro')

In [ ]:
# List the contents of the directory
!ls

In [ ]:
# Note the dimensions of our 3D MRI image - dim4 specifies the number of MRI volumes we have.
# Here it is 1 because we are viewing a structural image. For a functional dataset, this should be larger than 1.
# Similarly, the pixel dimensions specify the voxel sizes in mm.
# pixdim4 specifies the time in seconds to collect a single volume, the repetition time (TR).
!fslinfo highres

**Relevant information:**
- Number of voxels: dim1, dim2, dim3
- Voxel size: pixdim1, pixdim2, pixdim3

In [ ]:
# For even more information, we can read the full header of the NIFTI file.
!fslhd highres

In [ ]:
# fslstats can give us statistics about our image such as:
# the absolute range of intensities/robust range excluding outliers (-R/r),
# the mean intensity of all voxels/mean of non-zero voxels (-m/M),
# or number of voxels and volume/number of voxels and volume of non-zero voxels (-v/V)
!fslstats highres -R -r
!fslstats highres -M -m
!fslstats highres -V -v

In [ ]:
# Using fslmaths we can carry out basic operations on images,
# such as taking the difference of two images
!fslmaths image0 -sub image1 imdiff

In [ ]:
# We can also do more complex things like calculate
# percent signal change of our image relative to image0
!fslmaths imdiff -div image0 -mul 100 imdiffpercent

In [ ]:
# LThal_mask_func.nii.gz is an image where every voxel contains
# a probability (0 to 1) of that voxel belonging to the thalamus.
# We can threshold these probabilities and binarize the image to create a mask
!fslmaths LThal_mask_func -thr 0.5 -bin LThal_mask_func_bin

In [ ]:
# Next we could extract the size of this Region of Interest (ROI),
# the size of the overlap between this ROI and the thresholded statistic map,
# and finally the mean value of the statistics within this ROI.
!fslstats LThal_mask_func_bin -V
!fslstats thresh_zstat1 -k LThal_mask_func_bin -V
!fslstats thresh_zstat1 -k LThal_mask_func_bin -M

In [ ]:
# If you are ever unsure about the usage of a tool in FSL,
# try running without any inputs, like so:
!fslstats

We will learn more about FSL in the upcoming labs!

## III. Visualization

It's important to be able to visualize the result of operations we carry out on our brain images (after all, this is a Neuroimaging course). We have several options which we will use depending on convenience throughout the course labs.

The first option is called **FSLeyes** and can be locally installed relatively easily for macOS, Linux and Windows (via the Windows Subsystem for Linux), by following the instructions at the link provided. It is installed by default with FSL, but can be installed independently as well.

Another option which is mainly for Diffusion MRI is **MRview**.

For Python visualization options, see the examples below.

In [ ]:
from ipyniivue import AnyNiivue

nv1 = AnyNiivue()

# To visualize the Thalamus mask we used in the last step we can load
# the volume with the example_func volume underneath
volumes = [
    {"path": "example_func.nii.gz", "colormap": "gray"},
    {"path": "LThal_mask_func.nii.gz", "colormap": "red"}
]

# Load all the volumes into the viewer
nv1.load_volumes(volumes)
nv1

In [ ]:
nv2 = AnyNiivue()

# Let's view a higher resolution image for fun!
volumes = [
    {"path": "highres.nii.gz", "colormap": "gray"},
]

# Load all the volumes into the viewer
nv2.load_volumes(volumes)
nv2

## IV. Neuroimaging Tools in Python

Some useful python packages we may use in the labs can be found here.

**Nipy** is an ecosystem of python libraries for analyzing different types of brain data:

- **MNE** - Electrophysiology (MEG and EEG)
- **Nilearn** - Statistical analysis and Machine Learning with brain data
- **Nipy** - Structural and Functional MRI
- **Nibabel** - Useful tool for managing common brain data file formats (NIFTI, etc.)
- **Dipy** - Diffusion MRI

We will briefly touch upon some of these in later labs, but you are more than welcome to use any of these libraries for your projects.

## V. Final Exercise

**Objective:** Consolidate the use of FSL commands and the visualization of neuroimaging data.

**Tasks:**
1. Use `fslinfo` to identify the dimensions and voxel size of the image `structural.nii.gz`.
2. Create a binary mask of the thalamus with a threshold of 0.7 using the probability map `LThal_mask_func.nii.gz`.
3. Use `fslstats` to calculate the volume (in mm³) of this new binary mask.
4. Visualize your created mask overlaid on the functional image (`example_func.nii.gz`) using ipyniivue.

In [ ]:
# 1. Check structural image information
!fslinfo structural.nii.gz

In [ ]:
# 2. Create binary mask with a 0.7 threshold
# -thr 0.7: keeps only values > 0.7
# -bin: binarizes the remaining voxels to 1
!fslmaths LThal_mask_func.nii.gz -thr 0.7 -bin my_thalamus_mask.nii.gz

In [ ]:
# 3. Calculate volume statistics
# -V: output <voxels> <volume> (for nonzero voxels)
!fslstats my_thalamus_mask.nii.gz -V

In [ ]:
from ipyniivue import AnyNiivue

# 4. Visualize the result
nv_exercise = AnyNiivue()

# Define volumes: background functional image and the new mask
exercise_volumes = [
    {"path": "example_func.nii.gz", "colormap": "gray"},
    {"path": "my_thalamus_mask.nii.gz", "colormap": "blue"}
]

nv_exercise.load_volumes(exercise_volumes)
nv_exercise